In [2]:
from dotenv import load_dotenv
import os
import json
from pprint import pprint, pp
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters   import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import  HuggingFaceEmbeddings
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

from langchain_community.document_loaders import PyPDFLoader,TextLoader
from typing import IO, BinaryIO
from langchain_openai import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from uuid import uuid4
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# for Chroma 
import chromadb
from langchain_chroma import Chroma
# for FAISS
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore


In [3]:
%config IPCompleter.greedy=True

In [56]:
load_dotenv("../../../../../tokens/token_env")
#load_dotenv("langsmith_env")os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
#os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
#os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [7]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model='gpt-4o', temperature=0.2)

https://python.langchain.com/docs/integrations/tools/
https://github.com/sunnysavita10/Agentic-2.0/blob/main/langgraph/tools.ipynb

In [11]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [40]:
wiki_apiwrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=4000)

In [41]:
wikiquerytool=WikipediaQueryRun(api_wrapper=wiki_apiwrapper, handle_tool_error=True)

In [42]:
pp(wikiquerytool.description)

('A wrapper around Wikipedia. Useful for when you need to answer general '
 'questions about people, places, companies, facts, historical events, or '
 'other subjects. Input should be a search query.')


In [43]:
pp(wikiquerytool.args)

{'query': {'description': 'query to look up on wikipedia',
           'title': 'Query',
           'type': 'string'}}


In [47]:
#pprint(wikiquerytool.invoke({'query': 'Narendra Modi'}))

In [45]:
respose=wikiquerytool.invoke({'query': 'multimodal rag'})

In [46]:
print(respose)

Page: Large language model
Summary: A large language model (LLM) is a machine learning model designed for natural language processing tasks, especially language generation. LLMs are language models with many parameters, and are trained with self-supervised learning on a vast amount of text.
The largest and most capable LLMs are generative pretrained transformers (GPTs), which are largely used in generative chatbots such as ChatGPT or Gemini. LLMs can be fine-tuned for specific tasks or guided by prompt engineering. These models acquire predictive power regarding syntax, semantics, and ontologies inherent in human language corpora, but they also inherit inaccuracies and biases present in the data they are trained in.



Page: Prompt injection
Summary: Prompt injection is a cybersecurity exploit in which adversaries craft inputs that appear legitimate but are designed to cause unintended behavior in  machine learning models, particularly large language models (LLMs). This attack takes ad

In [48]:
from langchain_community.tools import YouTubeSearchTool

In [49]:
utube= YouTubeSearchTool()

In [51]:
utube('sunny savitha')

"['https://www.youtube.com/watch?v=_V3zqVlCACQ&pp=ygUNc3Vubnkgc2F2aXRoYQ%3D%3D', 'https://www.youtube.com/watch?v=6GnsndNL5u4&pp=ygUNc3Vubnkgc2F2aXRoYQ%3D%3D']"

In [53]:
utube('krish naik')

"['https://www.youtube.com/watch?v=JxgmHe2NyeY&pp=ygUKa3Jpc2ggbmFpaw%3D%3D', 'https://www.youtube.com/watch?v=p4pHsuEf4Ms&pp=ygUKa3Jpc2ggbmFpaw%3D%3D']"

In [54]:
from langchain_community.tools import TavilySearchResults


In [64]:
#to Crawl internet realtime use Tavilty
tsr=TavilySearchResults(max_results=3,topic="general")

In [65]:
tsr.invoke("what happened in rcb victory celebration")

[{'title': "Bengaluru stampede case: What we know so far on RCB's ...",
  'url': 'https://timesofindia.indiatimes.com/sports/cricket/ipl/top-stories/bengaluru-stampede-case-what-we-know-so-far-on-rcbs-celebrations-that-turned-catastrophic/articleshow/121670873.cms',
  'content': "[Follow us](https://news.google.com/publications/CAAqBwgKMM6y_Qowwu70Ag)\n\nRoyal Challengers Bengaluru's IPL victory celebrations turned tragic as a stampede at M Chinnaswamy Stadium resulted in 11 deaths and 75 injuries. FIRs have been filed against RCB, events organisers DNA, and Karnataka State Cricket Association for alleged negligence. The Karnataka High Court has sought a report from the government while arrests have been and officials suspended.\n\nRead More [...] [](https://timesofindia.indiatimes.com/city/bengaluru/celebrations-turn-tragic-bloodbath-at-rcbs-maiden-ipl-trophy-victory-day-to-remember-forever-see-pics/photostory/121646177.cms)[Celebrations turn tragic: Bloodbath at RCB’s maiden IPL trop

In [ ]:
#Custom tool
from langchain.tools import tool
@tool                                  ## use tool decorator 
def mutiply(a:int, b:int) -> int:
    """
     this tool will help multiply two numbers and return
    """
    return a * b

In [73]:
response=mutiply.invoke({'a':3, 'b':4 })   # when calling tool  use invoke and pass in dict, invoke will be part of chain so always call invoke when using chains

In [76]:
mutiply.name

'mutiply'

In [75]:
response

12

In [77]:
mutiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [98]:
@tool
def get_word_length(input: str) -> int:           ### tools always need to docstring 
    """
    Returns the length of the input word.

    Args:
        input (str): The input string.

    Returns:
        int: The length of the input string.
    """
    return len(input)

In [96]:
get_word_length("milind")

6

In [97]:
get_word_length.invoke(input="milind")

6